In [ ]:
import os
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from datetime import datetime
import time

In [ ]:
start_time = time.time()

# Using the token.json file to authenticate the user.
creds = Credentials.from_authorized_user_file('token.json', ['https://www.googleapis.com/auth/drive'])
# Building a service object that will be used to make requests to the Google Drive API.
service = build('drive', 'v3', credentials=creds)

FOLDER_ID = 'YOUR_FOLDER_ID'
PATH=''


# Create Drive API client
drive_service = build('drive', 'v3', credentials=creds)

In [ ]:
def get_directory_tree_and_files(folder_id, path='', results=[]):
    query = f"'{folder_id}' in parents and trashed = false"
    files = drive_service.files().list(q=query, fields='nextPageToken, files(id, name, mimeType, createdTime, modifiedTime, webViewLink, description, size, owners(emailAddress))').execute().get('files', [])
    for file in files:
        file_path = os.path.join(path, file['name'])
        file_type = 'Folder' if file['mimeType'] == 'application/vnd.google-apps.folder' else file['mimeType']
        file_date = datetime.fromisoformat(file['createdTime'][:-1]).strftime('%Y-%m-%d %H:%M:%S')
        file_updated = datetime.fromisoformat(file['modifiedTime'][:-1]).strftime('%Y-%m-%d %H:%M:%S')
        file_url = file['webViewLink'] if 'webViewLink' in file else ''
        file_description = file['description'] if 'description' in file else ''
        file_size = file['size'] if 'size' in file else ''
        file_owner_email = file['owners'][0]['emailAddress'] if 'owners' in file and file['owners'] else ''
        results.append([file_path, os.path.basename(file_path), file_type, file_date, file_url, file_updated, file_description, file_size, file_owner_email])
        if file_type == 'Folder':
            get_directory_tree_and_files(file['id'], file_path, results)
    return results

In [ ]:
# Get the list of paths of the directory tree and files
directory_tree_and_files = get_directory_tree_and_files(FOLDER_ID, path=PATH)

# Print the Full Path, Name, Type, Date, URL, Last Updated, Description, Size, and Owner Email of the files and folders
for file in directory_tree_and_files:
    print(file[0])

print(f"Time taken to get data: {time.time() - start_time} seconds")

In [ ]:
#  print length
print(len(directory_tree_and_files))

new_list = []
# remove duplicates if there is same path
for i in directory_tree_and_files:
    if i not in new_list:
        new_list.append(i)

# print length
print(len(new_list))

In [ ]:
# convert size to kilobytes
for i in new_list:
    if i[7]:
        i[7] = int(i[7]) / 1024


In [ ]:
import pandas as pd
# save the data to a csv file
df = pd.DataFrame(new_list, columns=['Full Path', 'Name', 'Type', 'Date', 'URL', 'Last Updated', 'Description', 'Size', 'Owner Email'])
df.to_csv('gdrive2.csv', index=False)

# save the data to a excel file
df = pd.DataFrame(new_list, columns=['Full Path', 'Name', 'Type', 'Date', 'URL', 'Last Updated', 'Description', 'Size', 'Owner Email'])
df.to_excel('gdrive2.xlsx', index=False)

